In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.

In [ ]:
# import libraries
try:
  %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

For this challenge, you will use the SMS Spam Collection dataset. The dataset has already been grouped into train data and test data.

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
# import data files
train_data= pd.read_csv(train_file_path,sep='\t',header=None) # for tabulator separated files sep='\t' required
#val_data = 20% of train_data
test_data = pd.read_csv(test_file_path,sep='\t',header=None)

Pre-processing for text classification

Standardization refers to preprocessing the text, typically to remove punctuation or HTML elements to simplify the dataset. Tokenization refers to splitting strings into tokens (for example, splitting a sentence into individual words, by splitting on whitespace). Vectorization refers to converting tokens into numbers so they can be fed into a neural network.

In [ ]:
# dataset pre-processing (standardize,tokenize and vectorize)
max_features= 10000
vectorize_layer = keras.layers.TextVectorization(output_mode='int',max_tokens=max_features)
# Make a text-only dataset (without labels), then call adapt
train_text = train_data.iloc[:,1]
vectorize_layer.adapt(train_text)
train_data_vectorized = vectorize_layer(train_text)
test_data_vectorized = vectorize_layer(test_data.iloc[:,1])
train_labels = train_data.iloc[:,0]
test_labels = test_data.iloc[:,0]
train_labels = train_labels.replace({'ham': 0, 'spam': 1})
test_labels = test_labels.replace({'ham': 0, 'spam': 1})

In [ ]:
# create a neural net classifier model
embedding_dim = 16
model = keras.Sequential()
model.add(keras.layers.Embedding(max_features,embedding_dim))
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(1, activation='sigmoid'))
model.summary()

model.compile(loss=keras.losses.BinaryCrossentropy(),
              optimizer='adam',
              metrics=[tf.metrics.BinaryAccuracy(threshold=0.5)])

In [ ]:
epochs = 40
history = model.fit(train_data_vectorized,train_labels,epochs=epochs)

In [ ]:
loss = history.history['loss']
plt.plot(range(epochs),loss)

In [ ]:
test_loss, test_accuracy = model.evaluate(test_data_vectorized, test_labels)

You should create a function called predict_message that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  vectorized_str= vectorize_layer(np.array([pred_text]))
  predicted = model.predict(vectorized_str).tolist()
  prediction = predicted[0]

  if prediction[0] <0.5:
    prediction.append('ham')
  else:
    prediction.append('spam')

  return (prediction)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
